In [0]:
#Loading up our drive on google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#importing relevant modules
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [0]:
#Cloning the gpt-2 model
!git clone https://github.com/tenoke/gpt-2

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
#changing the root directory
cd gpt-2

[Errno 2] No such file or directory: 'gpt-2'
/content/gpt-2


In [0]:
#Installing the required files in the requirements.txt file
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 604kB 8.2MB/s 
     |████████████████████████████████| 51kB 20.9MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=aa96c119a9b96b911dd30e93abc303a0793928713396332959f323b4160e87d7
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533181 sha256=53acd071c6364e687abb0c9352ace9428372d85e56e851c20ad5aa507cde6fd3
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
#downloading the model
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 662kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 36.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 737kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:24, 57.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.46Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 26.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 32.0Mit/s]                                                       


In [0]:
#setting up the kaggle api so that we can access our ubuntu corpus dataset
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ayobami00","key":"9bb7c11a1acbd33a0040e056d1b9afa5"}'}

In [0]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Oct 10 23:18 kaggle.json


In [0]:
#Installing the Kaggle API client
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

MessageError: ignored

In [0]:
#Downloading the dataset from kaggle
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

 99% 794M/799M [00:09<00:00, 134MB/s]
100% 799M/799M [00:09<00:00, 88.7MB/s]


In [0]:
!unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
#Reading the different datasets into pandas
dataset1 = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText.csv', parse_dates=['date'])

In [0]:
dataset1.head()

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23 14:55:00+00:00,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23 14:56:00+00:00,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23 14:57:00+00:00,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01 06:22:00+00:00,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01 06:22:00+00:00,mechtech,NaN,WTF?


In [0]:
dataset2 = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_196.csv', parse_dates=['date'])

In [0]:
dataset2.tail()

,folder,dialogueID,date,from,to,text
9212872,13,3676.tsv,2012-07-07 20:17:00+00:00,MonkeyDust,legolas,= arian
9212873,13,3676.tsv,2012-07-07 20:18:00+00:00,MonkeyDust,legolas,"observation and deduction, dear watson"
9212874,13,16586.tsv,2008-07-25 01:53:00+00:00,linuxfce,NaN,i am trying to install nvidia drivers from the...
9212875,13,16586.tsv,2008-07-25 01:53:00+00:00,linuxfce,NaN,how do i enter runlevel 3? when i try telinit ...
9212876,13,16586.tsv,2008-07-25 01:54:00+00:00,linuxfce,NaN,anyone know how to enter runlevel 3 in ubuntu?


In [0]:
dataset3 = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_301.csv', parse_dates=['date'])

In [0]:
dataset3.tail()

,folder,dialogueID,date,from,to,text
16587825,32,1783.tsv,2007-11-15 03:38:00+00:00,koyo001,NaN,thanks
16587826,32,1783.tsv,2007-11-15 03:39:00+00:00,koyo001,NaN,does anyone know something
16587827,32,1783.tsv,2007-11-15 03:39:00+00:00,neverblue,NaN,"no, no one knows everything"
16587828,32,1783.tsv,2007-11-15 03:40:00+00:00,koyo001,ikonia,the camera doesnt work
16587829,32,1783.tsv,2007-11-15 03:40:00+00:00,neverblue,koyo001,I believe you missed a post or two while you w...


In [0]:
#Merging the two datastes
combined = pd.merge(dataset1,dataset2,how = 'outer')

In [0]:
#Merging it with the third
final_data = pd.merge(combined,dataset3)

In [0]:
final_data.to_csv('./final_csv_data')

In [0]:
datasets = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_196.csv', parse_dates=['date'], chunksize=1200000)

In [0]:
#Pre-processing our data
i = 1

for dataset in datasets:
  dataset['date'] = dataset['date'].astype(int64) // 10**9
  
  text_corpus = ''
  current = None
  for msg in dataset.itertuples():
    if msg.dialogueID != current:
      current = msg.dialogueID
      text_corpus += '\n\n'
    try:
      text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
    except KeyError:
      pass
  
  with open(f'ubuntu-data/ubuntu-cleaned-{i}.txt', 'w') as f:
    f.write(text_corpus)
  del(text_corpus)
  i +=1
    
del(datasets)
del(dataset)

In [0]:
!du -h ubuntu-data/*

91M	ubuntu-data/ubuntu-cleaned-1.txt
92M	ubuntu-data/ubuntu-cleaned-2.txt
90M	ubuntu-data/ubuntu-cleaned-3.txt
89M	ubuntu-data/ubuntu-cleaned-4.txt
89M	ubuntu-data/ubuntu-cleaned-5.txt
86M	ubuntu-data/ubuntu-cleaned-6.txt
85M	ubuntu-data/ubuntu-cleaned-7.txt
60M	ubuntu-data/ubuntu-cleaned-8.txt


In [0]:
#Encoding our data
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-1.txt ubuntu-npz/ubuntu-cleaned-1.txt.npz --model_name 345M

Reading files
100% 1/1 [03:36<00:00, 216.66s/it]
Writing ubuntu-npz/ubuntu-cleaned-1.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-2.txt ubuntu-npz/ubuntu-cleaned-2.txt.npz --model_name 345M

Reading files
100% 1/1 [03:32<00:00, 212.22s/it]
Writing ubuntu-npz/ubuntu-cleaned-2.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-3.txt ubuntu-npz/ubuntu-cleaned-3.txt.npz --model_name 345M

Reading files
100% 1/1 [03:18<00:00, 198.31s/it]
Writing ubuntu-npz/ubuntu-cleaned-3.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-4.txt ubuntu-npz/ubuntu-cleaned-4.txt.npz --model_name 345M

Reading files
100% 1/1 [03:27<00:00, 207.94s/it]
Writing ubuntu-npz/ubuntu-cleaned-4.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-5.txt ubuntu-npz/ubuntu-cleaned-5.txt.npz --model_name 345M

Reading files
100% 1/1 [03:14<00:00, 194.02s/it]
Writing ubuntu-npz/ubuntu-cleaned-5.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-6.txt ubuntu-npz/ubuntu-cleaned-6.txt.npz --model_name 345M

Reading files
100% 1/1 [03:15<00:00, 195.86s/it]
Writing ubuntu-npz/ubuntu-cleaned-6.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-7.txt ubuntu-npz/ubuntu-cleaned-7.txt.npz --model_name 345M

Reading files
100% 1/1 [03:03<00:00, 183.55s/it]
Writing ubuntu-npz/ubuntu-cleaned-7.txt.npz


In [0]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-8.txt ubuntu-npz/ubuntu-cleaned-8.txt.npz --model_name 345M

Reading files
100% 1/1 [02:13<00:00, 133.68s/it]
Writing ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [0]:
!cp -r ubuntu-npz/ /content/drive/My\ Drive/ubuntu-npz/

In [0]:
!du -h ubuntu-npz/*

41M	ubuntu-npz/ubuntu-cleaned-1.txt.npz
46M	ubuntu-npz/ubuntu-cleaned-2.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-3.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-4.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-5.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-6.txt.npz
37M	ubuntu-npz/ubuntu-cleaned-7.txt.npz
27M	ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [0]:
#TRAINING OUR DATA AND GENERATING UNCONDITIONAL SAMPLES

#With default learnig rate
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --stop_after=1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-11 00:20:19.260601: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-11 00:20:19.261123: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e13480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-11 00:20:19.261174: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-11 00:20:19.293222: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-11 00:20:19.424125: I tensorflow/stream_executor/cu

In [0]:
#With learning rate of 0.0001
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --learning_rate 0.0001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-11 01:54:06.138686: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-11 01:54:06.139650: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x134b480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-11 01:54:06.139760: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-11 01:54:06.144903: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-11 01:54:06.231669: I tensorflow/stream_executor/cu